In [ ]:
import pandas as pd

# Чтение данных

In [ ]:
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")

# Разделение данных на train и val

In [ ]:
X = train_data.drop(columns = "label")
y = train_data["label"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
import torch


X_train = X_train.values
X_train = X_train.astype('float32')

X_train = torch.tensor(X_train)
X_train = X_train.reshape(-1, 1, 28, 28)

X_val = X_val.values
X_val = X_val.astype('float32')

X_val = torch.tensor(X_val)
X_val = X_val.reshape(-1, 1, 28, 28)\

X_test = test_data.values
X_test = X_test.astype('float32')

X_test = torch.tensor(X_test)
X_test = X_test.reshape(-1, 1, 28, 28)

In [ ]:
y_train = y_train.values
y_train = torch.tensor(y_train)

y_val = y_val.values
y_val = torch.tensor(y_val)

# Создание DataLoader

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test)

batch_size = 64

train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 64, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = False)

# Сверточная нейронная сеть

In [ ]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = 3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2)
        )
        self.flatten = nn.Flatten()
        self.out = nn.Linear(16 * 5 * 5, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.out(x)
        return x

# Обучение

In [ ]:
max_epochs = 10

In [ ]:
from tqdm import tqdm
def fit_one_epoch(model, train_loader, optimizer, criterion):
    sum_loss = 0
    for X_batch, y_batch in tqdm(train_loader):
        outp = model(X_batch)
        loss = criterion(outp, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        sum_loss += loss.item()
    return sum_loss/len(train_loader)
    

In [ ]:
def eval_one_epoch(model, val_loader, criterion):
    sum_loss = 0
    all_preds = 0
    true_preds =  0
    for X_batch, y_batch in tqdm(val_loader):
        with torch.no_grad():
            outp = model(X_batch)
            loss = criterion(outp, y_batch)
            sum_loss += loss.item()
            all_preds += len(y_batch)
            preds = torch.argmax(outp, dim = 1)
            true_preds += (preds == y_batch).sum()
    return sum_loss/len(val_loader), true_preds/all_preds

In [ ]:
def fit(model, train_loader, val_loader, optimizer, criterion, max_epochs):
    train_losses = []
    val_losses = []
    accuracies = []
    for i in range(max_epochs):
        train_losses.append(fit_one_epoch(model, train_loader, optimizer, criterion))
        print(f"Epoch: {i}")
        print(f"Train loss: {train_losses[-1]}")
        val_loss, acc = eval_one_epoch(model, val_loader, criterion)
        val_losses.append(val_loss)
        accuracies.append(acc)
        print(f"Val loss: {val_loss}")
        print(f"Accuracy: {acc}")
    return train_losses, val_losses, accuracies

In [ ]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
train_losses, val_losses, accuracies = fit(model, train_loader, val_loader, optimizer, criterion, max_epochs)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_losses, label = "Loss на обучающей выборке")
plt.plot(val_losses, label = "Loss на валидационной выборке")
plt.xlabel("Эпоха")
plt.ylabel("Loss")
plt.title("Кривая обучения")
plt.legend()
plt.show()

# Submit на Kaggle 

In [ ]:
outp = model(X_test)
preds = torch.argmax(outp, dim = 1)
preds.size()
submission = pd.DataFrame({
    "ImageId": range(1, len(preds) + 1),
    "Label": preds.numpy()
})

In [ ]:
submission.to_csv("submission.csv", index = False)

Accuracy на тестовой выборке: 0.97389